# Neural Graph Learning and Graph Regularization

In this tutorial, we will be creating a graph regularized version for a topic classification task. The task is to classify paper depending on their content. However in order to do so, we will also use the information encoded in the citation network that relates documents among each other. Of course, we do know that this kind of information is indeed powerful as papers belonging to the same subject tend to reference each other.   

### Load Dataset

For this tutorial we will be using the Cora dataset available in the stellargraph library 

In [1]:
from stellargraph import datasets

In [2]:
dataset = datasets.Cora()

In [3]:
%config Completer.use_jedi = False

In [4]:
dataset.download()

In [5]:
label_index = {
      'Case_Based': 0,
      'Genetic_Algorithms': 1,
      'Neural_Networks': 2,
      'Probabilistic_Methods': 3,
      'Reinforcement_Learning': 4,
      'Rule_Learning': 5,
      'Theory': 6,
  }

In [6]:
G, labels = dataset.load()

We now create the Dataset object where we will both include information of the targeted sample (node) and its neighbors. In the following we will also allow to control the number of labelling instances to be used, in order to reproduce and evaluate the classification performance in a semi-supervised setting. 

In [7]:
import numpy as np
from sklearn import preprocessing, feature_extraction, model_selection

In [8]:
import tensorflow as tf
from tensorflow.train import Example, Features, Feature, Int64List, BytesList, FloatList

In [9]:
GRAPH_PREFIX="NL_nbr"

In [10]:
def _int64_feature(*value):
    """Returns int64 tf.train.Feature from a bool / enum / int / uint."""
    return Feature(int64_list=Int64List(value=list(value)))

def _bytes_feature(value):
    """Returns bytes tf.train.Feature from a string."""
    return Feature(
        bytes_list=BytesList(value=[value.encode('utf-8')])
    )

def _float_feature(*value):
    return Feature(float_list=FloatList(value=list(value)))


In [11]:
from functools import reduce
from typing import List, Tuple
import pandas as pd
import six

def addFeatures(x, y):
    res = Features()
    res.CopyFrom(x)
    res.MergeFrom(y)
    return res

def neighborFeatures(features: Features, weight: float, prefix: str):
    data = {f"{prefix}_weight": _float_feature(weight)}
    for name, feature in six.iteritems(features.feature):
        data[f"{prefix}_{name}"] = feature 
    return Features(feature=data)

def neighborsFeatures(neighbors: List[Tuple[Features, float]]):
    return reduce(
        addFeatures, 
        [neighborFeatures(sample, weight, f"{GRAPH_PREFIX}_{ith}") for ith, (sample, weight) in enumerate(neighbors)],
        Features()
    )

def getNeighbors(idx, adjMatrix, topn=5):
    weights = adjMatrix.loc[idx]
    return weights[weights>0].sort_values(ascending=False).head(topn).to_dict()
    

def semisupervisedDataset(G, labels, ratio=0.2, topn=5):
    n = int(np.round(len(labels)*ratio))
    
    labelled, unlabelled = model_selection.train_test_split(
        labels, train_size=n, test_size=None, stratify=labels
    )
    
    adjMatrix = pd.DataFrame.sparse.from_spmatrix(G.to_adjacency_matrix(), index=G.nodes(), columns=G.nodes())
    
    features = pd.DataFrame(G.node_features(), index=G.nodes())
    
    dataset = {
        index: Features(feature = {
            #"id": _bytes_feature(str(index)), 
            "id": _int64_feature(index),
            "words": _float_feature(*[float(x) for x in features.loc[index].values]), 
            "label": _int64_feature(label_index[label])
        })
        for index, label in pd.concat([labelled, unlabelled]).items()
    }
    
    trainingSet = [
        Example(features=addFeatures(
            dataset[exampleId], 
            neighborsFeatures(
                [(dataset[nodeId], weight) for nodeId, weight in getNeighbors(exampleId, adjMatrix, topn).items()]
            )
        ))
        for exampleId in labelled.index
    ]
    
    testSet = [Example(features=dataset[exampleId]) for exampleId in unlabelled.index]

    serializer = lambda _list: [e.SerializeToString() for e in _list]
    
    return serializer(trainingSet), serializer(testSet)


We split the dataset into a training set and a test set

In [12]:
trainingSet, testSet = semisupervisedDataset(G, labels)

In [13]:
import tensorflow as tf

In [14]:
from tensorflow.data import Dataset

In [15]:
vocabularySize = 1433

In [16]:
neighbors=2
defaultWord = tf.constant(0, dtype=tf.float32, shape=[vocabularySize])

def parseExample(example, training=True):
    schema = {
        'words': tf.io.FixedLenFeature([vocabularySize], tf.float32, default_value=defaultWord),
        'label': tf.io.FixedLenFeature((), tf.int64, default_value=-1)
    }
    
    if training is True:
        for i in range(neighbors):
            name = f"{GRAPH_PREFIX}_{i}"
            schema[f"{name}_weight"] = tf.io.FixedLenFeature([1], tf.float32, default_value=[0.0])
            schema[f"{name}_words"] = tf.io.FixedLenFeature([vocabularySize], tf.float32, default_value=defaultWord)
    
    features = tf.io.parse_single_example(example, schema)
    
    label = features.pop("label")
    return features, label

In [17]:
def sampleGenerator(dataset):
    def wrapper():
        for example in dataset:
            yield example
    return wrapper
            
myTrain = Dataset \
    .from_generator(sampleGenerator(trainingSet), output_types=tf.string, output_shapes=()) \
    .map(lambda x: parseExample(x, True))

myTest = Dataset \
    .from_generator(sampleGenerator(testSet), output_types=tf.string, output_shapes=()) \
    .map(lambda x: parseExample(x, False))


In [19]:
for features, labels in myTrain.batch(10).take(1):
    print(features)
    print(labels)

{'NL_nbr_0_weight': <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[2.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [2.],
       [1.],
       [1.]], dtype=float32)>, 'NL_nbr_0_words': <tf.Tensor: shape=(10, 1433), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, 'NL_nbr_1_weight': <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>, 'NL_nbr_1_words': <tf.Tensor: shape=(10, 1433), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
     

In [20]:
for features, labels in myTest.batch(10).take(1):
    print(features)
    print(labels)

{'words': <tf.Tensor: shape=(10, 1433), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>}
tf.Tensor([1 1 3 3 2 4 5 3 2 6], shape=(10,), dtype=int64)


### Creating the model 

We now create the model that we will use to classify the documents

In [21]:
layers = [50, 50]

In [22]:
"""Creates a functional API-based multi-layer perceptron model."""
def create_model(num_units):
    inputs = tf.keras.Input(
          shape=(vocabularySize,), dtype='float32', name='words'
    )

    # outputs = tf.keras.layers.Dense(len(label_index), activation='softmax')(inputs)

    cur_layer =  inputs

    for num_units in layers:
        cur_layer = tf.keras.layers.Dense(num_units, activation='relu')(cur_layer)
        cur_layer = tf.keras.layers.Dropout(0.8)(cur_layer)

    outputs = tf.keras.layers.Dense(len(label_index), activation='softmax')(cur_layer)

    return tf.keras.Model(inputs, outputs=outputs)

In [23]:
from tensorflow.keras.callbacks import TensorBoard

#### Vanilla Model

We first train a simple, vanilla version that does not use the citation network information

In [24]:
model = create_model([50, 50])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [25]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
words (InputLayer)           [(None, 1433)]            0         
_________________________________________________________________
dense (Dense)                (None, 50)                71700     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 357       
Total params: 74,607
Trainable params: 74,607
Non-trainable params: 0
_________________________________________________________

In [25]:
model.fit(myTrain.batch(128), epochs=200, verbose=1, validation_data=myTest.batch(128),
          callbacks=[TensorBoard(log_dir='/tmp/noRegularization')])

Epoch 1/200


/Users/deusebio/.pyenv/versions/3.7.6/envs/ml-book-4/lib/python3.7/site-packages/tensorflow/python/keras/engine/functional.py:595: UserWarning: Input dict contained keys ['NL_nbr_0_weight', 'NL_nbr_0_words', 'NL_nbr_1_weight', 'NL_nbr_1_words'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


5/5 [==============================] - 2s 323ms/step - loss: 1.9820 - accuracy: 0.1390 - val_loss: 1.9311 - val_accuracy: 0.2022
Epoch 2/200
5/5 [==============================] - 0s 114ms/step - loss: 1.9699 - accuracy: 0.1779 - val_loss: 1.9219 - val_accuracy: 0.2373
Epoch 3/200
5/5 [==============================] - 0s 117ms/step - loss: 1.9632 - accuracy: 0.1964 - val_loss: 1.9135 - val_accuracy: 0.2742
Epoch 4/200
5/5 [==============================] - 0s 106ms/step - loss: 2.0003 - accuracy: 0.1981 - val_loss: 1.9071 - val_accuracy: 0.2886
Epoch 5/200
5/5 [==============================] - 1s 117ms/step - loss: 1.9681 - accuracy: 0.1810 - val_loss: 1.9010 - val_accuracy: 0.2941
Epoch 6/200
5/5 [==============================] - 0s 117ms/step - loss: 1.8860 - accuracy: 0.2400 - val_loss: 1.8951 - val_accuracy: 0.3010
Epoch 7/200
5/5 [==============================] - 1s 124ms/step - loss: 1.8809 - accuracy: 0.2603 - val_loss: 1.8888 - val_accuracy: 0.3052
Epoch 8/200
5/5 [========

Epoch 59/200
5/5 [==============================] - 1s 143ms/step - loss: 1.4438 - accuracy: 0.3667 - val_loss: 1.5221 - val_accuracy: 0.4280
Epoch 60/200
5/5 [==============================] - 0s 112ms/step - loss: 1.4592 - accuracy: 0.3633 - val_loss: 1.5149 - val_accuracy: 0.4326
Epoch 61/200
5/5 [==============================] - 1s 182ms/step - loss: 1.4056 - accuracy: 0.3988 - val_loss: 1.5073 - val_accuracy: 0.4367
Epoch 62/200
5/5 [==============================] - 1s 125ms/step - loss: 1.4253 - accuracy: 0.3948 - val_loss: 1.5000 - val_accuracy: 0.4414
Epoch 63/200
5/5 [==============================] - 1s 149ms/step - loss: 1.3744 - accuracy: 0.4412 - val_loss: 1.4925 - val_accuracy: 0.4460
Epoch 64/200
5/5 [==============================] - 1s 133ms/step - loss: 1.3908 - accuracy: 0.4052 - val_loss: 1.4845 - val_accuracy: 0.4478
Epoch 65/200
5/5 [==============================] - 1s 119ms/step - loss: 1.3819 - accuracy: 0.4087 - val_loss: 1.4763 - val_accuracy: 0.4538
Epoch 

5/5 [==============================] - 1s 241ms/step - loss: 0.8766 - accuracy: 0.6567 - val_loss: 1.5082 - val_accuracy: 0.5822
Epoch 174/200
5/5 [==============================] - 1s 219ms/step - loss: 0.7819 - accuracy: 0.7044 - val_loss: 1.5063 - val_accuracy: 0.5822
Epoch 175/200
5/5 [==============================] - 1s 184ms/step - loss: 0.8579 - accuracy: 0.6566 - val_loss: 1.5071 - val_accuracy: 0.5822
Epoch 176/200
5/5 [==============================] - 1s 186ms/step - loss: 0.8649 - accuracy: 0.6656 - val_loss: 1.5106 - val_accuracy: 0.5813
Epoch 177/200
5/5 [==============================] - 1s 193ms/step - loss: 0.8462 - accuracy: 0.6377 - val_loss: 1.5143 - val_accuracy: 0.5826
Epoch 178/200
5/5 [==============================] - 1s 175ms/step - loss: 0.8405 - accuracy: 0.6676 - val_loss: 1.5171 - val_accuracy: 0.5845
Epoch 179/200
5/5 [==============================] - 1s 183ms/step - loss: 0.8065 - accuracy: 0.6713 - val_loss: 1.5199 - val_accuracy: 0.5831
Epoch 180/200

#### Graph Regularized Version

We now create the graph-regularized version that uses the citation network information

In [26]:
base_model = create_model([50, 50])

In [27]:
import neural_structured_learning as nsl

In [34]:
graph_reg_config = nsl.configs.make_graph_reg_config(
    max_neighbors=2,
    multiplier=0.1,
    distance_type=nsl.configs.DistanceType.L2,
    sum_over_axis=-1)
graph_reg_model = nsl.keras.GraphRegularization(base_model,
                                                graph_reg_config)
graph_reg_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
#graph_reg_model.fit(train_dataset, epochs=200, verbose=1)

In [29]:
graph_reg_model.fit(myTrain.batch(128), epochs=200, verbose=1, validation_data=myTest.batch(128),
          callbacks=[TensorBoard(log_dir='/tmp/regularization')])

Epoch 1/200


/Users/deusebio/.pyenv/versions/3.7.6/envs/ml-book-4/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/GraphRegularization/graph_loss/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/GraphRegularization/graph_loss/Reshape:0", shape=(None, 7), dtype=float32), dense_shape=Tensor("gradient_tape/GraphRegularization/graph_loss/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


5/5 [==============================] - 4s 328ms/step - loss: 2.1052 - accuracy: 0.1049 - scaled_graph_loss: 0.0056 - val_loss: 1.9455 - val_accuracy: 0.1440
Epoch 2/200
5/5 [==============================] - 1s 179ms/step - loss: 2.0342 - accuracy: 0.1722 - scaled_graph_loss: 0.0047 - val_loss: 1.9367 - val_accuracy: 0.1934
Epoch 3/200
5/5 [==============================] - 1s 156ms/step - loss: 1.9881 - accuracy: 0.1603 - scaled_graph_loss: 0.0039 - val_loss: 1.9293 - val_accuracy: 0.2355
Epoch 4/200
5/5 [==============================] - 1s 178ms/step - loss: 1.9918 - accuracy: 0.1816 - scaled_graph_loss: 0.0035 - val_loss: 1.9224 - val_accuracy: 0.2770
Epoch 5/200
5/5 [==============================] - 1s 162ms/step - loss: 1.9551 - accuracy: 0.1983 - scaled_graph_loss: 0.0029 - val_loss: 1.9165 - val_accuracy: 0.2978
Epoch 6/200
5/5 [==============================] - 1s 156ms/step - loss: 1.9745 - accuracy: 0.1918 - scaled_graph_loss: 0.0028 - val_loss: 1.9113 - val_accuracy: 0.303

5/5 [==============================] - 1s 256ms/step - loss: 1.2262 - accuracy: 0.5307 - scaled_graph_loss: 0.0216 - val_loss: 1.3858 - val_accuracy: 0.5702
Epoch 98/200
5/5 [==============================] - 1s 197ms/step - loss: 1.2362 - accuracy: 0.5532 - scaled_graph_loss: 0.0216 - val_loss: 1.3793 - val_accuracy: 0.5725
Epoch 99/200
5/5 [==============================] - 1s 262ms/step - loss: 1.2081 - accuracy: 0.5314 - scaled_graph_loss: 0.0236 - val_loss: 1.3731 - val_accuracy: 0.5748
Epoch 100/200
5/5 [==============================] - 1s 160ms/step - loss: 1.2115 - accuracy: 0.5213 - scaled_graph_loss: 0.0224 - val_loss: 1.3678 - val_accuracy: 0.5780
Epoch 101/200
5/5 [==============================] - 1s 153ms/step - loss: 1.1994 - accuracy: 0.5480 - scaled_graph_loss: 0.0230 - val_loss: 1.3620 - val_accuracy: 0.5785
Epoch 102/200
5/5 [==============================] - 1s 144ms/step - loss: 1.1956 - accuracy: 0.5351 - scaled_graph_loss: 0.0240 - val_loss: 1.3569 - val_accurac

5/5 [==============================] - 1s 222ms/step - loss: 1.0264 - accuracy: 0.6045 - scaled_graph_loss: 0.0281 - val_loss: 1.2458 - val_accuracy: 0.6136
Epoch 146/200
5/5 [==============================] - 1s 181ms/step - loss: 0.9464 - accuracy: 0.6266 - scaled_graph_loss: 0.0315 - val_loss: 1.2463 - val_accuracy: 0.6136
Epoch 147/200
5/5 [==============================] - 1s 191ms/step - loss: 1.0403 - accuracy: 0.5913 - scaled_graph_loss: 0.0275 - val_loss: 1.2475 - val_accuracy: 0.6127
Epoch 148/200
5/5 [==============================] - 1s 231ms/step - loss: 1.0299 - accuracy: 0.6055 - scaled_graph_loss: 0.0302 - val_loss: 1.2493 - val_accuracy: 0.6127
Epoch 149/200
5/5 [==============================] - 1s 174ms/step - loss: 1.0777 - accuracy: 0.5722 - scaled_graph_loss: 0.0313 - val_loss: 1.2508 - val_accuracy: 0.6127
Epoch 150/200
5/5 [==============================] - 1s 265ms/step - loss: 1.0012 - accuracy: 0.6296 - scaled_graph_loss: 0.0288 - val_loss: 1.2516 - val_accur

Epoch 193/200
5/5 [==============================] - 1s 165ms/step - loss: 0.8634 - accuracy: 0.6420 - scaled_graph_loss: 0.0301 - val_loss: 1.2704 - val_accuracy: 0.6270
Epoch 194/200
5/5 [==============================] - 1s 164ms/step - loss: 0.8267 - accuracy: 0.6727 - scaled_graph_loss: 0.0314 - val_loss: 1.2641 - val_accuracy: 0.6274
Epoch 195/200
5/5 [==============================] - 1s 172ms/step - loss: 0.8430 - accuracy: 0.6941 - scaled_graph_loss: 0.0338 - val_loss: 1.2606 - val_accuracy: 0.6283
Epoch 196/200
5/5 [==============================] - 1s 177ms/step - loss: 0.8967 - accuracy: 0.6375 - scaled_graph_loss: 0.0320 - val_loss: 1.2575 - val_accuracy: 0.6316
Epoch 197/200
5/5 [==============================] - 1s 164ms/step - loss: 0.8748 - accuracy: 0.6446 - scaled_graph_loss: 0.0315 - val_loss: 1.2558 - val_accuracy: 0.6320
Epoch 198/200
5/5 [==============================] - 1s 190ms/step - loss: 0.9019 - accuracy: 0.6390 - scaled_graph_loss: 0.0323 - val_loss: 1.25